In [53]:
import sys
if 'ipykernel_launcher.py' not in sys.argv[0]:
    manual = False
    scenario = sys.argv[1]
else:
    manual = True
    scenario = 'intermodal'
    %matplotlib inline

In [54]:
import geopandas as gpd
import pandas as pd
import numpy as np

work_path = r'../../../'
quetzal_path = work_path + r'quetzal_santo_domingo/model/'

plot_path = work_path + r'plot/scenarios/'
import sys
sys.path.insert(0, '../../../quetzal/')

from quetzal.model import stepmodel

In [55]:
from shapely import geometry

In [56]:
sm =  stepmodel.read_zip(quetzal_path + scenario +  '/assigned.zip')

zones: 100%|██████████| 35/35 [00:11<00:00,  3.16it/s]           


In [57]:
sm.volumes.loc[sm.volumes['origin'] == sm.volumes['destination'], ['car', 'nocar']] = 0 

In [58]:
pt = list(sm.mode_nests.reset_index().set_index('root').loc['pt']['route_type'])
print(pt)

['express_bus', 'subway', 'concho', 'minibus', 'bus', 'gondola', 'tram']


In [59]:
from imp import reload
from quetzal.model import summarymodel
reload(summarymodel)

<module 'quetzal.model.summarymodel' from 'c:\\Users\\qchasserieau\\quetzal_quartz\\quetzal_santo_domingo\\notebooks\\transport\\../../../quetzal\\quetzal\\model\\summarymodel.py'>

In [60]:
sm.los['od_fares'] = [{} for i in sm.los.index]

In [61]:
sm.fare_rules

,fare_id,route_id
0,concho,CP_C100_1
1,concho,CP_C100_2
2,concho,CP_C101E_1
3,concho,CP_C101E_2
4,concho,CP_C102_1
...,...,...
614,opret,27_febrero
615,opret,ecologico
616,opret,ecologico
617,opret,mella


In [62]:
sm.loaded_links = sm.links.copy()
sm.summary_link_max(inplace=True)
sm.summary_link_sum(inplace=True)
sm.summary_path_sum(inplace=True, dense=True)
sm.summary_path_average(inplace=True, dense=True)
sm.summary_aggregated_path_average(inplace=True)
sm.summary_earning(inplace=True, dense=True)

In [63]:
sm.fare_attributes

,fare_id,price,transfers,agency_id
0,bus,15.0,0.0,intrant
1,concho,25.0,0.0,intrant
2,express_bus,30.0,0.0,intrant
3,f0,25.0,0.0,intrant
4,f1,40.0,1.0,intrant
5,f2,55.0,2.0,intrant
6,f3,70.0,3.0,intrant
7,fn,80.0,10.0,intrant
8,integrated,50.0,inf,intrant
9,minibus,25.0,0.0,intrant


In [64]:
sm.links.columns

Index(['a', 'trip_id', 'link_sequence', 'b', 'route_id', 'direction_id',
       'route_type', 'route_color', 'road_a', 'road_b', 'road_node_list',
       'road_link_list', 'road_length', 'geometry', 'headway', 'speed',
       'length', 'time', 'route_type_group', 'duration', 'cost', 'origin',
       'destination', 'agency_id', 'volume', 'boardings'],
      dtype='object')

In [65]:
def od_stack(
    self, 
    segments=['car', 'nocar'], 
    costs=['price', 'time', 'in_vehicle_time', 'in_vehicle_length', 'ntransfers']
):
    sm = self
    los = pd.concat([sm.pt_los, sm.car_los])
    los['mode'] = los['route_type']
    los.loc[los['route_type'].isin(pt), 'mode'] = 'pt'

    columns = []

    los = pd.merge(
        los, sm.volumes[['origin', 'destination'] +  segments], 
        on=['origin', 'destination'], suffixes=['_old', '']
    )
    

    for segment in segments:
        los[(segment, 'volume')] = los[(segment, 'probability')] * los[segment]


    for segment in segments:
        columns.append((segment, 'volume'))
        for service in costs:
            column = (segment, service)
            columns.append(column)
            los[column] = los[(segment, 'probability')] * los[service]

    df = los.groupby(['origin', 'destination', 'mode'])[columns].sum()      
    df.columns = pd.MultiIndex.from_tuples(df.columns)

    # add root (weighted mean of segments)

    df[('root', 'volume')] = sum([df[(segment, 'volume')].fillna(0) for segment in segments])

    weighted_sum = sum(
        [
            df[segment][costs].apply(lambda s: s * df[(segment, 'volume')]).fillna(0)
            for segment in segments
        ]      
    )

    for c in costs: 
        df[('root', c)] = (weighted_sum[c] / df[('root', 'volume')]).fillna(0)

    df.columns.names = 'segment', 'sum'
    return df

In [66]:
def municipio_stack(sm, *args, **kwargs):
    df = od_stack(sm, *args, **kwargs)
    right = sm.zones[['municipio']]
    proto = df.stack('segment')
    index_names = proto.index.names
    left = proto.reset_index()
    merged = pd.merge(left, right, left_on=['origin'], right_index=True
                     ).set_index(index_names).reset_index('mode').reset_index(drop=True)

    wm = lambda x: np.average(x, weights=merged.loc[x.index, 'volume'])
    # Groupby and aggregate with your dictionary:
    return merged.groupby(['municipio', 'mode']).agg(wm).stack()

In [67]:
sm.pt_los = sm.los.loc[sm.los['route_type'] != 'car']
sm.car_los = sm.los.loc[sm.los['route_type'] == 'car']
sm.stack_municipio = municipio_stack(sm)
sm.stack_municipio.name = 'average'

In [68]:
sm.stack_municipio.unstack()

in_vehicle_length  in_vehicle_time  ntransfers  \
municipio               mode                                                   
los alcarrizos          pt         10467.106801      1520.523050    1.255520   
                        walk           0.000000         0.000000    0.000000   
santo domingo de guzman pt          4358.729167       810.284601    0.622990   
                        walk           0.000000         0.000000    0.000000   
santo domingo este      pt          7653.952423      1382.565809    0.876918   
                        walk           0.000000         0.000000    0.000000   
santo domingo norte     pt          8191.649605      1408.204139    1.110517   
                        walk           0.000000         0.000000    0.000000   
santo domingo oeste     pt          5020.960750       940.597220    0.802585   
                        walk           0.000000         0.000000    0.000000   

                                  price         time      volume  
municipio               mode                                      
los alcarrizos          pt    20.124631  2619.826503  167.089279  
                        walk   4.951300   379.809758   24.875519  
santo domingo de guzman pt    18.342585  1701.321401   34.860708  
                        walk   1.612434   544.677904   63.566790  
santo domingo este      pt    20.532392  2626.938599   37.898098  
                        walk   2.127110   689.594899   72.447538  
santo domingo norte     pt    24.708555  2687.165982  192.471300  
                        walk   5.652147   313.206066   20.162604  
santo domingo oeste     pt    17.783656  1956.590303   33.275954  
                        walk   1.059688   652.639467   95.254519

# Buffer

In [69]:
def buffer_stack(sm, route_types=()):
    sm = sm.copy()
    r = range(1,11)
    
    heavy_nodes = set(sm.links.loc[sm.links['route_type'].isin(route_types)]['a'])
    for b in r:
        buffer = geometry.MultiPoint(list(sm.nodes.loc[heavy_nodes]['geometry'])).buffer(b * 100)

        def in_buffer_ratio(geometry):
            return (geometry.intersection(buffer).area) / geometry.area

        sm.zones['ib' + str(b * 100)] = sm.zones['geometry'].apply(in_buffer_ratio)
    right = sm.zones[['ib' + str(b * 100) for b in r]].apply(lambda c: c*sm.zones['pop'])
    right.columns = [i * 100 for i in list(r)]
    
    tot = sm.zones['pop'].sum()
    s = right.sum() / tot
    s.name = 'ratio'
    s.index.name = 'distance'
    return s

if True:
    sm.stack_buffer = buffer_stack(sm, route_types=['subway', 'tram', 'express_bus', 'gondola'])

In [70]:
sm.to_excel(quetzal_path + scenario + '/stacks.xlsx')

In [71]:
sm.stack_earning.sum()/2

4259579.524262475

In [72]:
sm.stack_earning

fare_id
opret      2.322495e+06
f1         0.000000e+00
f3         0.000000e+00
bus        4.045072e+05
f0         0.000000e+00
concho     7.376459e+05
f2         0.000000e+00
minibus    7.949310e+05
intrant    4.259580e+06
Name: sum, dtype: float64